<a href="https://colab.research.google.com/github/shuhbam199/FebGithub/blob/main/Airbnb_Modeling_and_Deployment__1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re
from datetime import datetime, date
pd.set_option('display.max_columns', None)
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack
from scipy.sparse import vstack
from scipy import sparse
from scipy.sparse import csr_matrix
from sklearn.preprocessing import OneHotEncoder
import pickle
from sklearn import metrics
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV # Import from model_selection
from sklearn.model_selection import RandomizedSearchCV # Import from model_selection
from sklearn.preprocessing import LabelBinarizer,LabelEncoder
from scipy.stats import randint as sp_randint
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
import pickle
import joblib

# New Section

In [ ]:
# we'll be only using train,sessions and test data

!gdown 1LhRSnImdRQNQJb-FYNbj8PKRQoxiSgeF
!gdown 1OnAoRjrd6MnEmfSSawvKRc0q4fFQXeeU
!gdown 1lCckrHM6gki5pILgazgtOWC7vSp4RXeL

Downloading...
From: https://drive.google.com/uc?id=1LhRSnImdRQNQJb-FYNbj8PKRQoxiSgeF
To: /content/train_users.csv
100% 24.9M/24.9M [00:00<00:00, 74.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OnAoRjrd6MnEmfSSawvKRc0q4fFQXeeU
To: /content/test_users.csv
100% 6.76M/6.76M [00:00<00:00, 37.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1lCckrHM6gki5pILgazgtOWC7vSp4RXeL
From (redirected): https://drive.google.com/uc?id=1lCckrHM6gki5pILgazgtOWC7vSp4RXeL&confirm=t&uuid=7af8b56d-fa5c-4661-beae-e97bfba11550
To: /content/sessions.csv
100% 632M/632M [00:06<00:00, 94.2MB/s]


In [ ]:
train_df=pd.read_csv("train_users.csv")
test_df=pd.read_csv("test_users.csv")
session_df=pd.read_csv("sessions.csv")

In [ ]:
print(train_df.shape)

print(train_df.columns)

train_df.head()

(213451, 16)
Index(['id', 'date_account_created', 'timestamp_first_active',
       'date_first_booking', 'gender', 'age', 'signup_method', 'signup_flow',
       'language', 'affiliate_channel', 'affiliate_provider',
       'first_affiliate_tracked', 'signup_app', 'first_device_type',
       'first_browser', 'country_destination'],
      dtype='object')


,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [ ]:
print(session_df.shape)

print("\nColumns:",session_df.columns)

session_df.head()

(10567737, 6)

Columns: Index(['user_id', 'action', 'action_type', 'action_detail', 'device_type',
       'secs_elapsed'],
      dtype='object')


,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,319.0
1,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,67753.0
2,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,301.0
3,d1mm9tcy42,search_results,click,view_search_results,Windows Desktop,22141.0
4,d1mm9tcy42,lookup,NaN,NaN,Windows Desktop,435.0


In [ ]:
#Checking null values
session_df.isnull().sum()/ len(session_df)

,0
user_id,0.003264
action,0.007535
action_type,0.106570
action_detail,0.106570
device_type,0.000000
secs_elapsed,0.012872


In [ ]:
# Dropping rows where user id is missing
print(session_df.shape)

session_df = session_df.dropna(subset = ['user_id'])

print(session_df.shape)

(10567737, 6)
(10533241, 6)


In [ ]:
session_df['user_id'].nunique()

135483

In [ ]:
session_df.groupby('user_id', as_index=False).agg(list)

,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,00023iyk9l,"[index, dashboard, header_userpic, dashboard, ...","[view, view, data, view, partner_callback, mes...","[view_search_results, dashboard, header_userpi...","[Mac Desktop, Mac Desktop, Mac Desktop, Mac De...","[20438.0, 787.0, 850.0, 934.0, nan, 129817.0, ..."
1,0010k6l0om,"[search_results, show, personalize, show, sear...","[click, view, data, nan, click, click, nan, da...","[view_search_results, p3, wishlist_content_upd...","[Mac Desktop, Mac Desktop, Mac Desktop, Mac De...","[1708.0, 21260.0, 1223.0, 26.0, 847.0, 1230.0,..."
2,001wyh0pz8,"[search, search, search, show, social_connecti...","[click, click, click, view, data, -unknown-, v...","[view_search_results, view_search_results, vie...","[Android App Unknown Phone/Tablet, Android App...","[622.0, 1813.0, 1507.0, 6327.0, 927.0, 142.0, ..."
3,0028jgx1x1,"[show, reviews, show, search, show, search, re...","[view, data, view, click, view, click, data, s...","[user_profile, listing_reviews, p3, view_searc...","[-unknown-, -unknown-, -unknown-, -unknown-, -...","[6162.0, 75.0, 86.0, 13710.0, 25217.0, 10989.0..."
4,002qnbzfs5,"[social_connections, payment_methods, create, ...","[data, -unknown-, -unknown-, view, data, data,...","[user_social_connections, -unknown-, -unknown-...","[iPhone, iPhone, iPhone, iPhone, iPhone, iPhon...","[17135.0, 711.0, 274.0, 179.0, 483.0, 1.0, 782..."
...,...,...,...,...,...,...
135478,zzxox7jnrx,"[identity, kba, kba_update, kba_update, popula...","[-unknown-, -unknown-, -unknown-, -unknown-, -...","[-unknown-, -unknown-, -unknown-, -unknown-, -...","[Windows Desktop, Windows Desktop, Windows Des...","[1338.0, 10115.0, 23802.0, 16951.0, 49938.0, 7..."
135479,zzy7t0y9cm,"[personalize, header_userpic, create, personal...","[data, data, submit, data, view, click, click,...","[wishlist_content_update, header_userpic, crea...","[Windows Desktop, Windows Desktop, Windows Des...","[501.0, 3671.0, nan, 42612.0, 697.0, 25616.0, ..."
135480,zzysuoqg6x,"[hosting_social_proof, create, header_userpic]","[-unknown-, submit, data]","[-unknown-, create_user, header_userpic]","[Windows Desktop, Windows Desktop, Windows Des...","[1533.0, nan, 198.0]"
135481,zzywmcn0jv,"[header_userpic, personalize, ajax_lwlb_contac...","[data, data, click, click, message_post, submi...","[header_userpic, wishlist_content_update, cont...","[Windows Desktop, Windows Desktop, Windows Des...","[3590.0, 72175.0, 280030.0, 1061.0, 75684.0, 3..."


In [ ]:
#https://stackoverflow.com/questions/34776651/concatenate-rows-of-pandas-dataframe-with-same-id

session_df_concat = session_df.groupby('user_id', as_index=False).agg(lambda x: x.tolist())

print(session_df_concat.shape)

session_df_concat.head()

(135483, 6)


,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,00023iyk9l,"[index, dashboard, header_userpic, dashboard, ...","[view, view, data, view, partner_callback, mes...","[view_search_results, dashboard, header_userpi...","[Mac Desktop, Mac Desktop, Mac Desktop, Mac De...","[20438.0, 787.0, 850.0, 934.0, nan, 129817.0, ..."
1,0010k6l0om,"[search_results, show, personalize, show, sear...","[click, view, data, nan, click, click, nan, da...","[view_search_results, p3, wishlist_content_upd...","[Mac Desktop, Mac Desktop, Mac Desktop, Mac De...","[1708.0, 21260.0, 1223.0, 26.0, 847.0, 1230.0,..."
2,001wyh0pz8,"[search, search, search, show, social_connecti...","[click, click, click, view, data, -unknown-, v...","[view_search_results, view_search_results, vie...","[Android App Unknown Phone/Tablet, Android App...","[622.0, 1813.0, 1507.0, 6327.0, 927.0, 142.0, ..."
3,0028jgx1x1,"[show, reviews, show, search, show, search, re...","[view, data, view, click, view, click, data, s...","[user_profile, listing_reviews, p3, view_searc...","[-unknown-, -unknown-, -unknown-, -unknown-, -...","[6162.0, 75.0, 86.0, 13710.0, 25217.0, 10989.0..."
4,002qnbzfs5,"[social_connections, payment_methods, create, ...","[data, -unknown-, -unknown-, view, data, data,...","[user_social_connections, -unknown-, -unknown-...","[iPhone, iPhone, iPhone, iPhone, iPhone, iPhon...","[17135.0, 711.0, 274.0, 179.0, 483.0, 1.0, 782..."


In [ ]:
# Function to convert list into strings

def list_to_str_action(action):

    """
    Function to convert list into strings

    parameters: action

    returns : action

    """
    action = [ str(i) for i in action ]

    action = [ re.sub('nan','',i) for i in action ]

    action = ','.join(action)

    return action

In [ ]:
session_df_concat['action'] = session_df_concat['action'].apply(list_to_str_action)

session_df_concat['action'].head()

,action
0,"index,dashboard,header_userpic,dashboard,callb..."
1,"search_results,show,personalize,show,search_re..."
2,"search,search,search,show,social_connections,i..."
3,"show,reviews,show,search,show,search,reviews,c..."
4,"social_connections,payment_methods,create,show..."


In [ ]:
import pandas as pd

df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 6]
})

df.apply(sum)

,0
A,6
B,15


In [ ]:
df

,A,B
0,1,4
1,2,5
2,3,6


In [ ]:
session_df_concat['action_type'] = session_df_concat['action_type'].apply(list_to_str_action)

session_df_concat['action_type'].head()

,action_type
0,"view,view,data,view,partner_callback,message_p..."
1,"click,view,data,,click,click,,data,view,partne..."
2,"click,click,click,view,data,-unknown-,view,-un..."
3,"view,data,view,click,view,click,data,submit,-u..."
4,"data,-unknown-,-unknown-,view,data,data,data,,..."


In [ ]:
session_df_concat['action_detail'] = session_df_concat['action_detail'].apply(list_to_str_action)

session_df_concat['action_detail'].head()

,action_detail
0,"view_search_results,dashboard,header_userpic,d..."
1,"view_search_results,p3,wishlist_content_update..."
2,"view_search_results,view_search_results,view_s..."
3,"user_profile,listing_reviews,p3,view_search_re..."
4,"user_social_connections,-unknown-,-unknown-,us..."


In [ ]:
# Function to convert list into strings

def list_to_str_device(device):

    """
    Function to convert list into strings

    parameters: device

    returns : device

    """

    device = [ str(i) for i in device ]

    device = [ re.sub('nan','',i) for i in device ]

    device = ','.join(set(device)) #keeping only unique devices that the user has used

    return device

In [ ]:
session_df_concat['device_type'] = session_df_concat['device_type'].apply(list_to_str_device)

session_df_concat['device_type'].head()

,device_type
0,"Mac Desktop,iPhone"
1,Mac Desktop
2,Android App Unknown Phone/Tablet
3,"-unknown-,Android Phone"
4,"-unknown-,iPhone"


In [ ]:
def total_session_length(times):
    return sum(float(t) for t in times if pd.notna(t) and str(t) != 'nan')

In [ ]:
session_df_concat['secs_elapsed'] = session_df_concat['secs_elapsed'].apply(total_session_length)

session_df_concat['secs_elapsed'].head()

,secs_elapsed
0,867896.0
1,586543.0
2,282965.0
3,297010.0
4,6487080.0


In [ ]:
print(session_df_concat.shape)

session_df_concat.head()

(135483, 6)


,user_id,action,action_type,action_detail,device_type,secs_elapsed
0,00023iyk9l,"index,dashboard,header_userpic,dashboard,callb...","view,view,data,view,partner_callback,message_p...","view_search_results,dashboard,header_userpic,d...","Mac Desktop,iPhone",867896.0
1,0010k6l0om,"search_results,show,personalize,show,search_re...","click,view,data,,click,click,,data,view,partne...","view_search_results,p3,wishlist_content_update...",Mac Desktop,586543.0
2,001wyh0pz8,"search,search,search,show,social_connections,i...","click,click,click,view,data,-unknown-,view,-un...","view_search_results,view_search_results,view_s...",Android App Unknown Phone/Tablet,282965.0
3,0028jgx1x1,"show,reviews,show,search,show,search,reviews,c...","view,data,view,click,view,click,data,submit,-u...","user_profile,listing_reviews,p3,view_search_re...","-unknown-,Android Phone",297010.0
4,002qnbzfs5,"social_connections,payment_methods,create,show...","data,-unknown-,-unknown-,view,data,data,data,,...","user_social_connections,-unknown-,-unknown-,us...","-unknown-,iPhone",6487080.0


In [ ]:
train_df.head()

,id,date_account_created,timestamp_first_active,date_first_booking,gender,age,signup_method,signup_flow,language,affiliate_channel,affiliate_provider,first_affiliate_tracked,signup_app,first_device_type,first_browser,country_destination
0,gxn3p5htnn,2010-06-28,20090319043255,NaN,-unknown-,NaN,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,NDF
1,820tgsjxq7,2011-05-25,20090523174809,NaN,MALE,38.0,facebook,0,en,seo,google,untracked,Web,Mac Desktop,Chrome,NDF
2,4ft3gnwmtx,2010-09-28,20090609231247,2010-08-02,FEMALE,56.0,basic,3,en,direct,direct,untracked,Web,Windows Desktop,IE,US
3,bjjt8pjhuk,2011-12-05,20091031060129,2012-09-08,FEMALE,42.0,facebook,0,en,direct,direct,untracked,Web,Mac Desktop,Firefox,other
4,87mebub9p4,2010-09-14,20091208061105,2010-02-18,-unknown-,41.0,basic,0,en,direct,direct,untracked,Web,Mac Desktop,Chrome,US


In [ ]:
# Join train and session df

train_merge = train_df.merge(session_df_concat,left_on = 'id', right_on='user_id', how='left')

print("Train  :",train_df.shape)

print("Session:",session_df_concat.shape)

print("Merge  :",train_merge.shape)

print("No of users in Train Data with session info:",train_merge.shape[0])

print("{} / {} = {}".format(train_merge.shape[0],train_df.shape[0],np.round((train_merge.shape[0]/train_df.shape[0]),2)))

Train  : (213451, 16)
Session: (135483, 6)
Merge  : (213451, 22)
No of users in Train Data with session info: 213451
213451 / 213451 = 1.0


only 35 % of users in train data have session info

In [ ]:
train_merge.isnull().sum()


,0
id,0
date_account_created,0
timestamp_first_active,0
date_first_booking,124543
gender,0
age,87990
signup_method,0
signup_flow,0
language,0
affiliate_channel,0


In [ ]:
print(test_df.shape)

print(test_df.columns)

test_df.head()

In [ ]:
# Join test and session df

test_merge = test_df.merge(session_df_concat, left_on='id', right_on='user_id', how='inner')

print("Test  :",test_df.shape)

print("Session:",session_df_concat.shape)

print("Merge  :",test_merge.shape)

print("No of users in Test Data with session info:",test_merge.shape[0])

print("{} / {} = {}".format(test_merge.shape[0],test_df.shape[0],np.round((test_merge.shape[0]/test_df.shape[0]),2)))

Test  : (62096, 15)
Session: (135483, 6)
Merge  : (61668, 21)
No of users in Test Data with session info: 61668
61668 / 62096 = 0.99


- Nearly 1 % of users in test data dont have session info.
- Thus we'll use left join.

In [ ]:
# Join test and session df

test_merge = test_df.merge(session_df_concat, left_on='id', right_on='user_id', how='left')

print("Test  :",test_df.shape)

print("Session:",session_df_concat.shape)

print("Merge  :",test_merge.shape)

print("No of users in Test Data with session info:",test_merge.shape[0])

print("{} / {} = {}".format(test_merge.shape[0],test_df.shape[0],np.round((test_merge.shape[0]/test_df.shape[0]),2)))

Test  : (62096, 15)
Session: (135483, 6)
Merge  : (62096, 21)
No of users in Test Data with session info: 62096
62096 / 62096 = 1.0


In [ ]:
#Checking null values
test_merge.isnull().sum()

In [ ]:
#fill

test_merge['user_id'].fillna('na' , inplace=True)

test_merge['action'].fillna('na' , inplace=True)

test_merge['action_type'].fillna('na' , inplace=True)

test_merge['action_detail'].fillna('na' , inplace=True)

test_merge['device_type'].fillna('na' , inplace=True)

test_merge['secs_elapsed'].fillna(0, inplace=True)

In [ ]:
test_merge.isnull().sum()

## Pre-processing final data

In [ ]:
train_merge.info()

In [ ]:
test_merge.info()

In [ ]:
# correcting the datatypes
train_merge['date_account_created'] = pd.to_datetime(train_merge['date_account_created'])
train_merge['age']= train_merge['age'].astype('Int64')

test_merge['date_account_created'] = pd.to_datetime(test_merge['date_account_created'])
test_merge['age']= test_merge['age'].astype('Int64')

In [ ]:
#Creating a new feature to reflect whether the user has made a booking or not
train_merge['has_booked'] = train_merge['date_first_booking'].notnull().astype(int)
train_merge.drop(columns=['date_first_booking'], inplace=True)

test_merge['has_booked'] = test_merge['date_first_booking'].notnull().astype(int)
test_merge.drop(columns=['date_first_booking'], inplace=True)

In [ ]:
train_merge['date_account_created_day'] = train_merge.date_account_created.dt.weekday

train_merge['date_account_created_month'] = train_merge.date_account_created.dt.month

train_merge['date_account_created_year'] = train_merge.date_account_created.dt.year

test_merge['date_account_created_day'] = test_merge.date_account_created.dt.weekday

test_merge['date_account_created_month'] = test_merge.date_account_created.dt.month

test_merge['date_account_created_year'] = test_merge.date_account_created.dt.year

In [ ]:
def func_timestamp_first_active(timestamp):

        """
        Function takes timestamp_first_active and converts it into datetime.

        Input type:  Integer

        return type: Datetime

        """

        timestamp = str(timestamp)

        timestamp = datetime(year=int(timestamp[0:4]), month=int(timestamp[4:6]), day=int(timestamp[6:8]),\

                             hour=int(timestamp[8:10]), minute=int(timestamp[10:12]), second=int(timestamp[12:]))

        return timestamp

In [ ]:
# daily,monthly,yearly and hourly data for timestamp_first_active

train_merge['timestamp_first_active'] = pd.to_datetime(train_merge.timestamp_first_active.apply(func_timestamp_first_active))

train_merge['timestamp_first_active_day'] = train_merge.timestamp_first_active.dt.weekday

train_merge['timestamp_first_active_month'] = train_merge.timestamp_first_active.dt.month

train_merge['timestamp_first_active_year'] = train_merge.timestamp_first_active.dt.year

train_merge['timestamp_first_active_hour'] = train_merge.timestamp_first_active.dt.hour

test_merge['timestamp_first_active'] = pd.to_datetime(test_merge.timestamp_first_active.apply(func_timestamp_first_active))

test_merge['timestamp_first_active_day'] = test_merge.timestamp_first_active.dt.weekday

test_merge['timestamp_first_active_month'] = test_merge.timestamp_first_active.dt.month

test_merge['timestamp_first_active_year'] = test_merge.timestamp_first_active.dt.year

test_merge['timestamp_first_active_hour'] = test_merge.timestamp_first_active.dt.hour

In [ ]:
# median replacement with median age = 34.0

def age_median(age):

    """
    Function to replace age outliers with median age

    parameters: age

    returns : age

    """

    if age < 15.0 or age > 100.0:

        return 34.0

    else:

        return age

In [ ]:
train_merge['age'] = train_merge['age'].fillna(-1)

train_merge['age'] = train_merge['age'].apply(age_median)

test_merge['age'] = test_merge['age'].fillna(-1)

test_merge['age'] = test_merge['age'].apply(age_median)

In [ ]:
#creating age buckets

bins = [i for i in range(15,106,5)]

def age_interv(age):

    """

    Function takes age and returns interval.

    parameters:  age

    returns: age interval

    """

    for i in range(len(bins)):

        if age < bins[i]:

            return i

In [ ]:
train_merge['age_interv'] = train_merge['age'].apply(lambda x: age_interv(x))

test_merge['age_interv'] = test_merge['age'].apply(lambda x: age_interv(x))

In [ ]:
# mode replacement for first_affiliate_tracked

train_merge.first_affiliate_tracked.fillna('untracked',inplace = True)

test_merge.first_affiliate_tracked.fillna('untracked',inplace = True)

In [ ]:
train_merge.gender.replace('-unknown-','OTHER', inplace=True)

test_merge.gender.replace('-unknown-','OTHER', inplace=True)

In [ ]:
#getting labels

y = train_merge['country_destination']

In [ ]:
# drop colmns

train_merge.drop(['id','date_account_created','timestamp_first_active','country_destination','user_id'],axis=1,inplace = True)

test_merge.drop(['id','date_account_created','timestamp_first_active','user_id'],axis=1,inplace = True)

In [ ]:
# OHE

# https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data

lst_ohe_train = []

ohe = ['gender', 'signup_method', 'language', 'affiliate_channel',\
            'affiliate_provider', 'first_affiliate_tracked', 'signup_app', 'first_device_type', 'first_browser']

for feat in ohe:

    train_ohe = pd.get_dummies(train_merge[feat], prefix=feat)

    train_merge.drop([feat], axis=1, inplace = True)

    test_ohe = pd.get_dummies(test_merge[feat], prefix=feat)

    test_merge.drop([feat], axis=1, inplace = True)

    lst_ohe_train.append(train_ohe.columns)

    # Get missing columns in the training test

    missing_cols = set( train_ohe.columns ) - set( test_ohe.columns )

    # Add a missing column in test set with default value equal to 0

    for c in missing_cols:

        test_ohe[c] = 0

    # Ensure the order of column in the test set is in the same order than in train set

    test_ohe = test_ohe[train_ohe.columns]

    train_merge = pd.concat((train_merge, train_ohe), axis=1)

    test_merge = pd.concat((test_merge, test_ohe), axis=1)

In [ ]:
# OHE device_type

# https://stackoverflow.com/questions/57469676/python-one-hot-encoding-for-comma-separated-values

train_ohe = train_merge['device_type'].str.get_dummies(sep=",")

train_merge.drop(['device_type'], axis=1, inplace = True)

test_ohe = test_merge['device_type'].str.get_dummies(sep=",")

test_merge.drop(['device_type'], axis=1, inplace = True)

# Get missing columns in the training test

missing_cols = set( train_ohe.columns ) - set( test_ohe.columns )

lst_ohe_train.append(train_ohe.columns)

# Add a missing column in test set with default value equal to 0

for c in missing_cols:

    test_ohe[c] = 0

# Ensure the order of column in the test set is in the same order than in train set

test_ohe = test_ohe[train_ohe.columns]

train_merge = pd.concat((train_merge, train_ohe), axis=1)

test_merge = pd.concat((test_merge, test_ohe), axis=1)

In [ ]:
train_merge.head()

,age,signup_flow,action,action_type,action_detail,secs_elapsed,has_booked,date_account_created_day,date_account_created_month,date_account_created_year,timestamp_first_active_day,timestamp_first_active_month,timestamp_first_active_year,timestamp_first_active_hour,age_interv,gender_FEMALE,gender_MALE,gender_OTHER,signup_method_basic,signup_method_facebook,signup_method_google,language_ca,language_cs,language_da,language_de,language_el,language_en,language_es,language_fi,language_fr,language_hr,language_hu,language_id,language_is,language_it,language_ja,language_ko,language_nl,language_no,language_pl,language_pt,language_ru,language_sv,language_th,language_tr,language_zh,affiliate_channel_api,affiliate_channel_content,affiliate_channel_direct,affiliate_channel_other,affiliate_channel_remarketing,affiliate_channel_sem-brand,affiliate_channel_sem-non-brand,affiliate_channel_seo,affiliate_provider_baidu,affiliate_provider_bing,affiliate_provider_craigslist,affiliate_provider_daum,affiliate_provider_direct,affiliate_provider_email-marketing,affiliate_provider_facebook,affiliate_provider_facebook-open-graph,affiliate_provider_google,affiliate_provider_gsp,affiliate_provider_meetup,affiliate_provider_naver,affiliate_provider_other,affiliate_provider_padmapper,affiliate_provider_vast,affiliate_provider_wayn,affiliate_provider_yahoo,affiliate_provider_yandex,first_affiliate_tracked_linked,first_affiliate_tracked_local ops,first_affiliate_tracked_marketing,first_affiliate_tracked_omg,first_affiliate_tracked_product,first_affiliate_tracked_tracked-other,first_affiliate_tracked_untracked,signup_app_Android,signup_app_Moweb,signup_app_Web,signup_app_iOS,first_device_type_Android Phone,first_device_type_Android Tablet,first_device_type_Desktop (Other),first_device_type_Mac Desktop,first_device_type_Other/Unknown,first_device_type_SmartPhone (Other),first_device_type_Windows Desktop,first_device_type_iPad,first_device_type_iPhone,first_browser_-unknown-,first_browser_AOL Explorer,first_browser_Android Browser,first_browser_Apple Mail,first_browser_Arora,first_browser_Avant Browser,first_browser_BlackBerry Browser,first_browser_Camino,first_browser_Chrome,first_browser_Chrome Mobile,first_browser_Chromium,first_browser_CometBird,first_browser_Comodo Dragon,first_browser_Conkeror,first_browser_CoolNovo,first_browser_Crazy Browser,first_browser_Epic,first_browser_Firefox,first_browser_Flock,first_browser_Google Earth,first_browser_Googlebot,first_browser_IE,first_browser_IE Mobile,first_browser_IceDragon,first_browser_IceWeasel,first_browser_Iron,first_browser_Kindle Browser,first_browser_Maxthon,first_browser_Mobile Firefox,first_browser_Mobile Safari,first_browser_Mozilla,first_browser_NetNewsWire,first_browser_OmniWeb,first_browser_Opera,first_browser_Opera Mini,first_browser_Opera Mobile,first_browser_Outlook 2007,first_browser_PS Vita browser,first_browser_Pale Moon,first_browser_Palm Pre web browser,first_browser_RockMelt,first_browser_Safari,first_browser_SeaMonkey,first_browser_Silk,first_browser_SiteKiosk,first_browser_SlimBrowser,first_browser_Sogou Explorer,first_browser_Stainless,first_browser_TenFourFox,first_browser_TheWorld Browser,first_browser_Yandex.Browser,first_browser_wOSBrowser,-unknown-,Android App Unknown Phone/Tablet,Android Phone,Blackberry,Chromebook,Linux Desktop,Mac Desktop,Opera Phone,Tablet,Windows Desktop,Windows Phone,iPad Tablet,iPhone,iPodtouch
0,34.0,0,NaN,NaN,NaN,NaN,0,0,6,2010,3,3,2009,4,4,False,False,True,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,Fal

In [ ]:
#Checking null values
train_merge.isnull().sum()

,0
age,0
signup_flow,0
action,139636
action_type,139636
action_detail,139636
...,...
Windows Desktop,0
Windows Phone,0
iPad Tablet,0
iPhone,0


In [ ]:
#Checking null values in test data
test_merge.isnull().sum()

In [ ]:
print(train_merge.shape,y.shape)
print(test_merge.shape)

In [ ]:
#Count vectorizer for action feature

# Create Count Vectorizer
count_vec_action = CountVectorizer(tokenizer=lambda x: x.split(','))

#building vocabulary using train data
count_vec_action.fit(train_merge['action'].values)

#vectorization on train and test data
train_merge_action_count_vec=count_vec_action.transform(train_merge['action'].values)
test_merge_action_count_vec=count_vec_action.transform(test_merge['action'].values)

print("After vectorizations")
print(train_merge_action_count_vec.shape)
print(test_merge_action_count_vec.shape)

ValueError: np.nan is an invalid document, expected byte or unicode string.

In [ ]:
#looking at the features
action_feat=count_vec_action.get_feature_names_out()

action_feat

NotFittedError: Vocabulary not fitted or provided

In [ ]:
train_merge_action_count_vec_dense=train_merge_action_count_vec.toarray() #converting to dense matrix

test_merge_action_count_vec_dense=test_merge_action_count_vec.toarray() #converting to dense matrix

train_merge_action_count_vec_df = pd.DataFrame(train_merge_action_count_vec_dense, columns=action_feat) #converting it to a df for merging

test_merge_action_count_vec_df = pd.DataFrame(test_merge_action_count_vec_dense, columns=action_feat) #converting it to a df for merging

print(train_merge_action_count_vec_df.shape)

print(test_merge_action_count_vec_df.shape)

In [ ]:
#Count vectorizer for action type feature

# Create Count Vectorizer
count_vec_action_type = CountVectorizer(tokenizer=lambda x: x.split(','))

#building vocabulary using train data
count_vec_action_type.fit(train_merge['action_type'].values)

#vectorization on train and test data
train_merge_action_type_count_vec=count_vec_action_type.transform(train_merge['action_type'].values)
test_merge_action_type_count_vec=count_vec_action_type.transform(test_merge['action_type'].values)

print("After vectorizations")
print(train_merge_action_type_count_vec.shape)
print(test_merge_action_type_count_vec.shape)

In [ ]:
#looking at the features
action_type_feat=count_vec_action_type.get_feature_names_out()

action_type_feat

In [ ]:
train_merge_action_type_count_vec_dense=train_merge_action_type_count_vec.toarray() #converting to dense matrix

test_merge_action_type_count_vec_dense=test_merge_action_type_count_vec.toarray() #converting to dense matrix

train_merge_action_type_count_vec_df = pd.DataFrame(train_merge_action_type_count_vec_dense, columns=action_type_feat) #converting it to a df for merging

test_merge_action_type_count_vec_df = pd.DataFrame(test_merge_action_type_count_vec_dense, columns=action_type_feat) #converting it to a df for merging

print(train_merge_action_type_count_vec_df.shape)

print(test_merge_action_type_count_vec_df.shape)

In [ ]:
#Count vectorizer for action detail feature

# Create Count Vectorizer
count_vec_action_detail = CountVectorizer(tokenizer=lambda x: x.split(','))

#building vocabulary using train data
count_vec_action_detail.fit(train_merge['action_detail'].values)

#vectorization on train and test data
train_merge_action_detail_count_vec=count_vec_action_detail.transform(train_merge['action_detail'].values)
test_merge_action_detail_count_vec=count_vec_action_detail.transform(test_merge['action_detail'].values)

print("After vectorizations")
print(train_merge_action_detail_count_vec.shape)
print(test_merge_action_detail_count_vec.shape)

In [ ]:
#looking at the features
action_detail_feat=count_vec_action_detail.get_feature_names_out()

action_detail_feat

In [ ]:
train_merge_action_detail_count_vec_dense=train_merge_action_detail_count_vec.toarray() #converting to dense matrix

test_merge_action_detail_count_vec_dense=test_merge_action_detail_count_vec.toarray() #converting to dense matrix

train_merge_action_detail_count_vec_df = pd.DataFrame(train_merge_action_detail_count_vec_dense, columns=action_detail_feat) #converting it to a df for merging

test_merge_action_detail_count_vec_df = pd.DataFrame(test_merge_action_detail_count_vec_dense, columns=action_detail_feat) #converting it to a df for merging

print(train_merge_action_detail_count_vec_df.shape)

print(test_merge_action_detail_count_vec_df.shape)

In [ ]:
train_merge.drop(['action','action_type','action_detail'],axis=1,inplace = True)

test_merge.drop(['action','action_type','action_detail'],axis=1,inplace = True)

In [ ]:
train_merge_final_df= pd.concat([train_merge,train_merge_action_count_vec_df, train_merge_action_type_count_vec_df,train_merge_action_detail_count_vec_df], axis=1)

test_merge_final_df= pd.concat([test_merge,test_merge_action_count_vec_df, test_merge_action_type_count_vec_df,test_merge_action_detail_count_vec_df], axis=1)

In [ ]:
print(train_merge_final_df.shape)
print(test_merge_final_df.shape)

In [ ]:
# creating a list to store all the column names
col_lst = [i for i in train_merge.columns]

col_lst

In [ ]:
col_lst.extend(action_feat) #extend the list by adding new elements into it

col_lst.extend(action_type_feat)

col_lst.extend(action_detail_feat)

len(col_lst)

In [ ]:
le = LabelEncoder() #making the true lables numeric for ML modelling

y = le.fit_transform(y)

y

In [ ]:
y.shape

We'll need to save the test data, along with the label encoder to get the countries back from their labels along with that we'll be saving the best model.



In [ ]:
#saving train dataset
train_merge_final_df.to_csv("/content/final_train_dataset.csv", index=False)

#saving the test dataset
test_merge_final_df.to_csv("/content/final_test_dataset.csv", index=False)

#saving the label encoder
pickle.dump(le, open("/content/label_encoder.pickle", "wb"))

In [ ]:
# https://www.kaggle.com/davidgasquez/ndcg-scorer

def dcg_score(y_true, y_score, k=5):

    """Discounted cumulative gain (DCG) at rank K.

    Parameters
    ----------
    y_true : array, shape = [n_samples]
        Ground truth (true relevance labels).
    y_score : array, shape = [n_samples, n_classes]
        Predicted scores.
    k : int
        Rank.

    Returns
    -------
    score : float
    """

    order = np.argsort(y_score)[::-1] # sorts the indices of y_score in descending order. This helps us to rank the predictions from the highest to the lowest score.
    y_true = np.take(y_true, order[:k]) #reorders y_true based on the sorted indices and selects the top-k values. This aligns the true labels with the top-k predicted scores.

    gain = 2 ** y_true - 1 #calculate gain or numerator part of the dcg formula

    discounts = np.log2(np.arange(len(y_true)) + 2) #calculate discount or denominator part of the dcg formula
    return np.sum(gain / discounts)


def ndcg_score(ground_truth, predictions, k=5):

    """Normalized discounted cumulative gain (NDCG) at rank K.

    Normalized Discounted Cumulative Gain (NDCG) measures the performance of a
    recommendation system based on the graded relevance of the recommended
    entities. It varies from 0.0 to 1.0, with 1.0 representing the ideal
    ranking of the entities.

    Parameters
    ----------
    ground_truth : array, shape = [n_samples]
        Ground truth (true labels represended as integers).
    predictions : array, shape = [n_samples, n_classes]
        Predicted probabilities.
    k : int
        Rank.

    Returns
    -------
    score : float

    Example
    -------
    >>> ground_truth = [1, 0, 2]
    >>> predictions = [[0.15, 0.55, 0.2], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    1.0
    >>> predictions = [[0.9, 0.5, 0.8], [0.7, 0.2, 0.1], [0.06, 0.04, 0.9]]
    >>> score = ndcg_score(ground_truth, predictions, k=2)
    0.6666666666
    """

    lb = LabelBinarizer() #Initializes a binarizer to convert multi-class labels to binary indicators.
    lb.fit(range(predictions.shape[1] + 1)) # predictions.shape[1] equals number of classes it'll be 12 in this case
    T = lb.transform(ground_truth) #it'll reshape each ground truth lable to number of classes that is 5 since we are predicting for 5 countries
    print(f'The original ground truth shape is{ground_truth.shape}')
    print(f'The transformed ground truth shape is{T.shape}')
    scores = []

    # Iterate over each y_true and compute the DCG score
    for y_true, y_score in zip(T, predictions):
        actual = dcg_score(y_true, y_score, k) #to compute the DCG score for the actual predictions.
        best = dcg_score(y_true, y_true, k) #to compute the DCG score for the ideal predictions (i.e., using the true labels as predictions).
        score = float(actual) / float(best)
        scores.append(score)

    return np.mean(scores) #Returns the mean of the NDCG scores for all samples.


# NDCG Scorer function
ndcg_scorer = make_scorer(ndcg_score, needs_proba=True, k=5)

## Testing with Logistic regression

In [ ]:
params =  {'C':[0.0001, 0.001, 0.01]}

lr = linear_model.LogisticRegression(multi_class="multinomial",solver="lbfgs")

clf = RandomizedSearchCV(lr, params, verbose=10, n_iter=3 ,n_jobs=-1,scoring=ndcg_scorer)

In [ ]:
clf.fit(train_merge_final_df,y)

In [ ]:
clf.best_params_

In [ ]:
pred_y = clf.predict_proba(train_merge_final_df)

In [ ]:
pred_y.shape

the shape of pred_y is number_of_samples, num_of_classes here as we know that there are 12 unique destination countries our model gives probability scores for each of the countries since it's a multi-class classification task.

In [ ]:
#Train ndcg score

s = ndcg_score(y, pred_y, k=5)

s

## Testing with RF

In [ ]:
# training of the model takes time please do not run unless requried
params = {
'n_estimators' : [200, 700, 800, 1000, 1200],
'max_depth' : [15, 20, 25, 30, 35, 50],
'min_samples_split' : [2, 3, 5, 8],
'min_samples_leaf' : [1, 2, 5, 10] }

rf = RandomForestClassifier()

clf = RandomizedSearchCV(rf, params, verbose=10,n_jobs=-1,scoring=ndcg_scorer)

In [ ]:
clf.fit(train_merge_final_df,y) #takes more than 2 and half hour to run

In [ ]:
pred_y = clf.predict_proba(train_merge_final_df)

In [ ]:
clf.best_params_

In [ ]:
# Access the best estimator from the RandomizedSearchCV object
best_clf = clf.best_estimator_

In [ ]:
# Now we can access the feature importances
importances = best_clf.feature_importances_
indices = (np.argsort(importances))[-25:]
plt.figure(figsize=(10,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='r', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
#Train ndcg score

s = ndcg_score(y, pred_y, k=5)

s

Testing on test data

In [ ]:
test_id = test_df['id'].values

In [ ]:
pred = clf.predict_proba(test_merge_final_df)

ids = []
countries = []

# Taking the 5 classes with highest probabilities

for i in range(len(test_id)):
    idx = test_id[i]
    ids += [idx] * 5
    countries += le.inverse_transform(np.argsort(pred[i])[::-1][:5]).tolist()

# Generate submission

sub = pd.DataFrame({"id" : ids,"country" : countries})

In [ ]:
#saving ml model
pickle.dump(best_clf, open("/content/best_ml_model.pickle", "wb"))

## Testing with XgBoost [Homework]

In [ ]:
param_grid = {
'max_depth': sp_randint(3, 20),
'learning_rate': [0.001, 0.01, 0.1, 0.2],
'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
'min_child_weight': [0.25,0.5, 1.0, 3.0, 5.0, 7.0],
'gamma': [0, 0.25, 0.3,0.35,0.45,0.5,0.6,0.8,1.0],
'reg_lambda': [0.1,0.2,0.4,0.5,0.6,0.8,1.0,10.0],
'n_estimators':[100,200,500,1000,2000],
'colsample_bytree':[0.1,0.3,0.5,1],
'colsample_bylevel':[0.1,0.3,0.5,1]
}

gb = xgb.XGBClassifier(objective='multi:softmax',eval_metric= 'mlogloss')

clf = RandomizedSearchCV(gb, param_grid,n_jobs=-1,verbose=10,scoring=ndcg_scorer, random_state=42)

In [ ]:
clf.fit(train_merge_final_df,y)

In [ ]:
pred_y = clf.predict_proba(train_merge_final_df)

In [ ]:
# Train ndcg score

s = ndcg_score(y, pred_y, k=5)

s

In [ ]:
features = col_lst
importances = clf.feature_importances_
indices = (np.argsort(importances))[-25:]
plt.figure(figsize=(10,12))
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='r', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
test_id = test_df['id'].values

In [ ]:
pred = clf.predict_proba(test_merge_final_df)

ids = []
countries = []

# Taking the 5 classes with highest probabilities

for i in range(len(test_id)):
    idx = test_id[i]
    ids += [idx] * 5
    countries += le.inverse_transform(np.argsort(pred[i])[::-1][:5]).tolist()

# Generate submission

sub = pd.DataFrame({"id" : ids,"country" : countries})

## Deployment code

In [ ]:
import streamlit as st
import pandas as pd
import joblib

# Load the test dataset
test_df = pd.read_csv('final_test_dataset.csv')

# Load the trained ML model
model = joblib.load('best_ml_model.pkl')

# Define a function to predict the top 5 countries
def predict_top_5_countries(user_id, model, test_df):
    user_data = test_df[test_df['user_id'] == user_id]
    predictions = model.predict_proba(user_data)
    top_5_indices = predictions.argsort()[0][-5:][::-1]
    top_5_countries = [model.classes_[index] for index in top_5_indices]
    return top_5_countries

# Streamlit app
st.title('Top 5 Travel Destination Prediction')

# Dropdown menu for selecting Gender
user_id = st.selectbox('Select a Gender:', test_df['Gender'].unique())

if st.button('Predict'):
    top_5_countries = predict_top_5_countries(user_id, model, test_df)
    st.write(f'Top 5 predicted countries for user {user_id}:')
    for country in top_5_countries:
        st.write(country)

# Adding additional features to make the app more interesting
st.sidebar.title('Additional Features')
if st.sidebar.checkbox('Show User Details'):
    user_details = test_df[test_df['user_id'] == user_id]
    st.sidebar.write('User Details:')
    st.sidebar.write(user_details)

In [ ]:
from collections import counter

def process_data(mynos):